In [1]:
from methods import *

In [2]:
(
    m,
    loader,
    optim,
    model,
    num_domain,
    num_group,
    lr_scheduler,
    device,
    n,
    num_feature,
    num_class,
) = exp_init(
    'waterbirds',
    128,
    'y',
    'a',
    0,
    0,
    'fairness',
    0,
    True,
    '%s/privateDemographics/models/waterbirds/erm_num_epoch_360_batch_size_128_lr_1e-05_subsample_False_weight_decay_1_best.model' % root_dir,
    123,
    'erm',
    torch.device('cpu'),
)

Representations are loaded from folder /mnt/cloud/yzeng58/privateDemographics/data/waterbirds_resnet50_representation!


In [3]:
results = get_domain_george(
    m,
    loader,
    num_domain,
    num_class,
    4,
    device,
    123,
    'waterbirds',
    False,
)

Loading Representation for val set: 100%|██████████| 10/10 [00:00<00:00, 283.76it/s]


UMAP dimension reduction...
Clustering superclass 0...


TypeError: '<' not supported between instances of 'int' and 'str'